In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import lightgbm as lgb

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
data = pd.read_csv('./data/train_data.csv')

In [ ]:
data.head()